In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Tuple, Optional, Union
from pathlib import Path

In [11]:
# FARS + CRSS -> company summaries -> full census with metrics + simple exposure score + safety index
# Exposure fields (your names):
#   RECENT_MILEAGE  (miles)
#   NBR_POWER_UNIT  (trucks)
#   DRIVER_TOTAL    (drivers)

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Tuple, Optional, Union

# -------------------- IO & Normalization Helpers --------------------

def _read_parquet_required(path: Union[str, Path]) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Missing required file: {p.resolve()}")
    return pd.read_parquet(p)

def _stdcols(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = pd.Index([str(c).strip().upper() for c in d.columns])
    return d

def _clean_usdot_series(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip().str.replace(r"^\s*0+(?=\d)", "", regex=True)
    return s.mask(s.isin(["", "nan", "<NA>"]))

def _force_string_identifiers(df: pd.DataFrame) -> pd.DataFrame:
    id_tokens = ("VIN", "PLATE", "MCARR_I2", "DOT_NUMBER", "USDOT")
    d = df.copy()
    for c in d.columns:
        if any(tok in c.upper() for tok in id_tokens):
            d[c] = d[c].astype("string")
    return d

# -------------------- Standardization for merged crash rows --------------------

def standardize_census_fields(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expected columns for merged crash rows:
      DOT_NUMBER, LEGAL_NAME, DBA_NAME, SOURCE, LIKELY_AT_FAULT, FAULT_SCORE
    """
    d = _stdcols(df)

    if "DOT_NUMBER" not in d.columns and "MCARR_I2" in d.columns:
        d["DOT_NUMBER"] = d["MCARR_I2"]
    if "DOT_NUMBER" not in d.columns:
        raise ValueError("DOT_NUMBER not found (and no MCARR_I2 to backfill).")
    d["DOT_NUMBER"] = _clean_usdot_series(d["DOT_NUMBER"])

    if "LEGAL_NAME" not in d.columns:
        for alt in ("LEGALNAME", "CARRIER_LEGAL_NAME"):
            if alt in d.columns:
                d["LEGAL_NAME"] = d[alt]
                break
    if "LEGAL_NAME" not in d.columns:
        d["LEGAL_NAME"] = pd.NA

    if "DBA_NAME" not in d.columns:
        for alt in ("DBANAME", "CARRIER_DBA_NAME"):
            if alt in d.columns:
                d["DBA_NAME"] = d[alt]
                break
    if "DBA_NAME" not in d.columns:
        d["DBA_NAME"] = pd.NA

    for req in ("SOURCE", "LIKELY_AT_FAULT", "FAULT_SCORE"):
        if req not in d.columns:
            raise ValueError(f"Missing required field: {req}")

    d["LIKELY_AT_FAULT"] = pd.to_numeric(d["LIKELY_AT_FAULT"], errors="coerce")
    d["FAULT_SCORE"]     = pd.to_numeric(d["FAULT_SCORE"], errors="coerce")

    return _force_string_identifiers(d)

# -------------------- Full census backbone (2M+ rows) --------------------

def standardize_census_backbone(df: pd.DataFrame) -> pd.DataFrame:
    """
    Minimal standardization for the RAW census snapshot (ALL carriers).
    Ensures DOT_NUMBER, LEGAL_NAME, DBA_NAME exist; preserves exposure fields.
    """
    d = _stdcols(df)

    if "DOT_NUMBER" not in d.columns and "MCARR_I2" in d.columns:
        d["DOT_NUMBER"] = d["MCARR_I2"]
    if "DOT_NUMBER" not in d.columns:
        raise ValueError("Raw census snapshot missing DOT_NUMBER/MCARR_I2.")
    d["DOT_NUMBER"] = _clean_usdot_series(d["DOT_NUMBER"])

    if "LEGAL_NAME" not in d.columns:
        for alt in ("LEGALNAME", "CARRIER_LEGAL_NAME"):
            if alt in d.columns:
                d["LEGAL_NAME"] = d[alt]
                break
    if "LEGAL_NAME" not in d.columns:
        d["LEGAL_NAME"] = pd.NA

    if "DBA_NAME" not in d.columns:
        for alt in ("DBANAME", "CARRIER_DBA_NAME"):
            if alt in d.columns:
                d["DBA_NAME"] = d[alt]
                break
    if "DBA_NAME" not in d.columns:
        d["DBA_NAME"] = pd.NA

    return _force_string_identifiers(d)

# -------------------- Exposure helpers & rates (uses your field names) --------------------

def _first_existing(df: pd.DataFrame, candidates) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _num_series(df: pd.DataFrame, col: Optional[str]) -> pd.Series:
    if col is None or col not in df.columns:
        return pd.Series(np.nan, index=df.index)
    return pd.to_numeric(df[col], errors="coerce")

def add_exposure_rates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add exposure-adjusted crash rates and simple ratios to a census+metrics dataframe.
    Uses your fields first: RECENT_MILEAGE, NBR_POWER_UNIT, DRIVER_TOTAL.

    New columns added:
      - accidents_per_truck
      - accidents_per_driver
      - accidents_per_million_miles

    Existing (kept / created):
      - rate_per_100_trucks, rate_at_fault_per_100_trucks
      - rate_per_100_drivers, rate_at_fault_per_100_drivers
      - rate_per_1m_miles,   rate_at_fault_per_1m_miles
      - fars_rate_per_*, crss_rate_per_*
    """
    out = df.copy()

    pu_col    = _first_existing(out, ["NBR_POWER_UNIT","POWER_UNITS","POWER_UNITS_CNT","POWER_UNIT","PU","TRUCKS"])
    drv_col   = _first_existing(out, ["DRIVER_TOTAL","DRIVERS","DRIVERCNT","TOTAL_DRIVERS"])
    miles_col = _first_existing(out, ["RECENT_MILEAGE","MCS150_MILES","MCS150MILES","TOTAL_MILES","VMT","ANNUAL_VMT"])

    pu    = _num_series(out, pu_col).where(lambda s: s > 0)
    drv   = _num_series(out, drv_col).where(lambda s: s > 0)
    miles = _num_series(out, miles_col).where(lambda s: s > 0)

    tot    = pd.to_numeric(out.get("total_crashes", np.nan), errors="coerce")
    tot_af = pd.to_numeric(out.get("total_at_fault_crashes", np.nan), errors="coerce")

    # ---- Per-100 / per-1M (total & at-fault) ----
    out["rate_per_100_trucks"]           = (tot    / pu    * 100).where(pu.notna())
    out["rate_at_fault_per_100_trucks"]  = (tot_af / pu    * 100).where(pu.notna())

    out["rate_per_100_drivers"]          = (tot    / drv   * 100).where(drv.notna())
    out["rate_at_fault_per_100_drivers"] = (tot_af / drv   * 100).where(drv.notna())

    out["rate_per_1m_miles"]             = (tot    / miles * 1_000_000).where(miles.notna())
    out["rate_at_fault_per_1m_miles"]    = (tot_af / miles * 1_000_000).where(miles.notna())

    # ---- Simple ratios (at-fault) ----
    out["accidents_per_truck"]         = (tot_af / pu).where(pu.notna())
    out["accidents_per_driver"]        = (tot_af / drv).where(drv.notna())
    out["accidents_per_million_miles"] = out["rate_at_fault_per_1m_miles"]

    # ---- Source-specific rates (FARS/CRSS) ----
    for src in ("fars", "crss"):
        src_tot = pd.to_numeric(out.get(f"{src}_total", np.nan), errors="coerce")
        if src_tot.notna().any():
            out[f"{src}_rate_per_100_trucks"]  = (src_tot / pu    * 100).where(pu.notna())
            out[f"{src}_rate_per_100_drivers"] = (src_tot / drv   * 100).where(drv.notna())
            out[f"{src}_rate_per_1m_miles"]    = (src_tot / miles * 1_000_000).where(miles.notna())

    return out

# -------------------- Combined exposure score + Safety Index --------------------

def add_simple_exposure_score(
    df: pd.DataFrame,
    miles_scale: float = 1_000_000,
    safety_cap_pct: float = 99.0
) -> pd.DataFrame:
    """
    exposure_score = trucks + drivers + miles / miles_scale
    accident_exposure_score = total_at_fault_crashes / exposure_score
    safety_index = 100 * (1 - accident_exposure_score / pXX_cap), clipped to [0,100]
      where pXX_cap is the safety_cap_pct percentile of accident_exposure_score (default 99th)
    """
    out = df.copy()

    trucks  = pd.to_numeric(out.get("NBR_POWER_UNIT"),  errors="coerce").astype("float64")
    drivers = pd.to_numeric(out.get("DRIVER_TOTAL"),    errors="coerce").astype("float64")
    miles   = pd.to_numeric(out.get("RECENT_MILEAGE"),  errors="coerce").astype("float64")

    exposure = trucks.fillna(0.0) + drivers.fillna(0.0) + (miles.fillna(0.0) / float(miles_scale))
    exposure = exposure.where(exposure > 0)

    out["exposure_score"] = exposure.astype("float64")

    at_fault = pd.to_numeric(out.get("total_at_fault_crashes"), errors="coerce").astype("float64")
    score = (at_fault / out["exposure_score"]).replace([np.inf, -np.inf], np.nan).astype("float64")
    out["accident_exposure_score"] = score

    # Safety Index (0–100, higher = safer), capped by chosen percentile to avoid outlier collapse
    if score.notna().any():
        cap = np.nanpercentile(score, safety_cap_pct)
        denom = cap if np.isfinite(cap) and cap > 0 else np.nanmax(score.values)
        if denom and denom > 0:
            si = 100 * (1 - (score / denom))
            out["safety_index"] = np.clip(si, 0, 100)
        else:
            out["safety_index"] = np.nan
    else:
        out["safety_index"] = np.nan

    return out

# -------------------- Build per-source & overall summaries from merged rows --------------------

def build_company_summary(
    fars_merged_path: Union[str, Path] = "census_with_fars.parquet",
    crss_merged_path: Union[str, Path] = "census_with_crss.parquet",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Input: FARS- and CRSS-merged-to-census parquet files.
    Output:
      - per_source: DOT x SOURCE aggregates
      - overall: DOT-level aggregates (with fars_* and crss_* columns)
      - both: stacked crash rows (FARS + CRSS)
    """
    fars = standardize_census_fields(_read_parquet_required(fars_merged_path))
    crss = standardize_census_fields(_read_parquet_required(crss_merged_path))

    both = pd.concat([fars, crss], ignore_index=True)
    both = both[both["DOT_NUMBER"].notna()].copy()

    both["MATCHED_FLAG"] = (both["SOURCE"].notna()).astype(int)
    both["AT_FAULT_FLAG"] = np.where(
        both["MATCHED_FLAG"] == 1,
        (both["LIKELY_AT_FAULT"] == 1).astype("float"),
        np.nan,
    )

    per_source = (
        both.groupby(["DOT_NUMBER", "LEGAL_NAME", "DBA_NAME", "SOURCE"], dropna=False)
            .agg(
                total_crashes          = ("MATCHED_FLAG", "sum"),
                total_at_fault_crashes = ("AT_FAULT_FLAG", "sum"),
                pct_at_fault           = ("AT_FAULT_FLAG", "mean"),
                mean_fault_score       = ("FAULT_SCORE", "mean"),
            )
            .reset_index()
    )
    per_source["total_at_fault_crashes"] = per_source["total_at_fault_crashes"].fillna(0).astype(int)

    overall = (
        both.groupby(["DOT_NUMBER", "LEGAL_NAME", "DBA_NAME"], dropna=False)
            .agg(
                total_crashes          = ("MATCHED_FLAG", "sum"),
                total_at_fault_crashes = ("AT_FAULT_FLAG", "sum"),
                mean_fault_score       = ("FAULT_SCORE", "mean"),
            )
            .reset_index()
    )
    overall["total_at_fault_crashes"] = overall["total_at_fault_crashes"].fillna(0).astype(int)

    fars_split = per_source.loc[
        per_source["SOURCE"] == "FARS",
        ["DOT_NUMBER", "total_crashes", "total_at_fault_crashes", "pct_at_fault"],
    ]
    fars_split.columns = ["DOT_NUMBER", "fars_total", "fars_at_fault", "fars_pct_at_fault"]

    crss_split = per_source.loc[
        per_source["SOURCE"] == "CRSS",
        ["DOT_NUMBER", "total_crashes", "total_at_fault_crashes", "pct_at_fault"],
    ]
    crss_split.columns = ["DOT_NUMBER", "crss_total", "crss_at_fault", "crss_pct_at_fault"]

    overall = (
        overall
        .merge(fars_split, on="DOT_NUMBER", how="left")
        .merge(crss_split, on="DOT_NUMBER", how="left")
    )

    for col in ["fars_total", "fars_at_fault", "crss_total", "crss_at_fault"]:
        overall[col] = overall[col].fillna(0).astype(int)

    overall["pct_at_fault"] = np.where(
        overall["total_crashes"] > 0,
        overall["total_at_fault_crashes"] / overall["total_crashes"],
        np.nan,
    )

    column_order = [
        "DOT_NUMBER", "LEGAL_NAME", "DBA_NAME",
        "total_crashes", "total_at_fault_crashes", "pct_at_fault",
        "mean_fault_score",
        "fars_total", "fars_at_fault", "fars_pct_at_fault",
        "crss_total", "crss_at_fault", "crss_pct_at_fault",
    ]
    remaining = [c for c in overall.columns if c not in column_order]
    overall = overall[column_order + remaining]

    overall = overall.sort_values(
        ["total_at_fault_crashes", "total_crashes", "pct_at_fault"],
        ascending=[False, False, False],
    ).reset_index(drop=True)

    return per_source, overall, both

# -------------------- Full census with metrics (matched-only fallback) --------------------

def make_full_census_with_metrics(combined_rows: pd.DataFrame, overall: pd.DataFrame) -> pd.DataFrame:
    """
    Build a census-with-metrics frame from merged crash rows (FARS+CRSS).
    """
    crash_cols = {"SOURCE", "LIKELY_AT_FAULT", "FAULT_SCORE", "MATCHED_FLAG", "AT_FAULT_FLAG"}
    census_cols = [c for c in combined_rows.columns if c not in crash_cols]
    census_backbone = combined_rows[census_cols].drop_duplicates().copy()

    metric_cols = [
        "DOT_NUMBER",
        "total_crashes", "total_at_fault_crashes", "pct_at_fault", "mean_fault_score",
        "fars_total", "fars_at_fault", "fars_pct_at_fault",
        "crss_total", "crss_at_fault", "crss_pct_at_fault",
    ]
    metrics = overall[metric_cols].copy()

    out = census_backbone.merge(metrics, on="DOT_NUMBER", how="left")

    for col in [
        "total_crashes", "total_at_fault_crashes",
        "fars_total", "fars_at_fault",
        "crss_total", "crss_at_fault",
    ]:
        if col in out.columns:
            out[col] = out[col].fillna(0).astype(int)

    out = add_exposure_rates(out)
    out = add_simple_exposure_score(out)  # adds exposure_score, accident_exposure_score, safety_index
    return out

# -------------------- Preferred: build full census from raw backbone --------------------

def make_full_census_with_metrics_from_base(
    census_base_path: Union[str, Path],
    overall: pd.DataFrame,
) -> pd.DataFrame:
    """
    Build census-with-metrics from a raw census backbone parquet (ALL carriers).
    """
    base = _read_parquet_required(census_base_path)
    base = standardize_census_backbone(base)

    metric_cols = [
        "DOT_NUMBER",
        "total_crashes", "total_at_fault_crashes", "pct_at_fault", "mean_fault_score",
        "fars_total", "fars_at_fault", "fars_pct_at_fault",
        "crss_total", "crss_at_fault", "crss_pct_at_fault",
    ]
    metrics = overall[metric_cols].copy()

    out = base.merge(metrics, on="DOT_NUMBER", how="left")

    for col in [
        "total_crashes", "total_at_fault_crashes",
        "fars_total", "fars_at_fault",
        "crss_total", "crss_at_fault",
    ]:
        if col in out.columns:
            out[col] = out[col].fillna(0).astype(int)

    out = add_exposure_rates(out)
    out = add_simple_exposure_score(out)  # adds exposure_score, accident_exposure_score, safety_index
    return out

# -------------------- Metrics-only export (single file) --------------------

def export_metrics_from_census_csv(
    overall: pd.DataFrame,
    census_csv_path: Union[str, Path],
    output_path: Union[str, Path] = "fars_crss_metrics_only.parquet",
) -> pd.DataFrame:
    """
    Build a DOT-level metrics table from:
      - `overall`: crash metrics per DOT (from build_company_summary)
      - `census_csv_path`: raw census CSV with exposure fields

    Writes a single file (Parquet by default) at `output_path` with:
      - DOT_NUMBER
      - exposure fields: NBR_POWER_UNIT, DRIVER_TOTAL, RECENT_MILEAGE
      - all accident / rate / safety metrics
    and returns the resulting DataFrame.
    """
    # 1) Load census CSV and standardize
    census_csv_path = Path(census_csv_path)
    base = pd.read_csv(census_csv_path, low_memory=False)
    base = standardize_census_backbone(base)

    # 2) Attach crash metrics from `overall`
    metric_cols = [
        "DOT_NUMBER",
        "total_crashes", "total_at_fault_crashes", "pct_at_fault", "mean_fault_score",
        "fars_total", "fars_at_fault", "fars_pct_at_fault",
        "crss_total", "crss_at_fault", "crss_pct_at_fault",
    ]
    metrics = overall[metric_cols].copy()

    out = base.merge(metrics, on="DOT_NUMBER", how="left")

    # Fill counts with zeros where appropriate
    for col in [
        "total_crashes", "total_at_fault_crashes",
        "fars_total", "fars_at_fault",
        "crss_total", "crss_at_fault",
    ]:
        if col in out.columns:
            out[col] = out[col].fillna(0).astype(int)

    # 3) Add exposure-based rates & scores
    out = add_exposure_rates(out)
    out = add_simple_exposure_score(out)  # exposure_score, accident_exposure_score, safety_index

    # 4) Keep only DOT + exposure + accident/rate/safety columns
    cols_to_keep = [
        "DOT_NUMBER",
        # crash counts & fault metrics
        "total_crashes", "total_at_fault_crashes",
        "pct_at_fault", "mean_fault_score",
        "fars_total", "fars_at_fault", "fars_pct_at_fault",
        "crss_total", "crss_at_fault", "crss_pct_at_fault",
        # exposure-adjusted rates
        "rate_per_100_trucks", "rate_at_fault_per_100_trucks",
        "rate_per_100_drivers", "rate_at_fault_per_100_drivers",
        "rate_per_1m_miles", "rate_at_fault_per_1m_miles",
        # simple ratios
        "accidents_per_truck", "accidents_per_driver", "accidents_per_million_miles",
        # source-specific rates
        "fars_rate_per_100_trucks", "fars_rate_per_100_drivers", "fars_rate_per_1m_miles",
        "crss_rate_per_100_trucks", "crss_rate_per_100_drivers", "crss_rate_per_1m_miles",
        # composite scores
        "exposure_score", "accident_exposure_score", "safety_index",
    ]
    cols_to_keep = [c for c in cols_to_keep if c in out.columns]

    metrics_only = out[cols_to_keep].copy()
    metrics_only = _force_string_identifiers(metrics_only)

    # 5) Export a single file
    output_path = Path(output_path)
    metrics_only.to_parquet(output_path, index=False)

    print("\nSaved metrics-only crash+exposure table:")
    print(" ", output_path.resolve())

    return metrics_only


In [13]:
# 1. Build crash summaries from your FARS/CRSS + census merged files
per_source, overall, combined_rows = build_company_summary(
    fars_merged_path="census_with_fars.parquet",      # update if different
    crss_merged_path="census_with_crss.parquet",
)

# 2. Create the single DOT-level metrics file using your CSV census
metrics_only = export_metrics_from_census_csv(
    overall=overall,
    census_csv_path="SMS_Input_-_Motor_Carrier_Census_Information_20250919.csv",
    output_path="fars_crss_census.parquet",
)

metrics_only.head()


Saved metrics-only crash+exposure table:
  /Users/bens1858/Desktop/Columbia/Fall 2025/Capstone/fars_crss_census.parquet


,DOT_NUMBER,total_crashes,total_at_fault_crashes,pct_at_fault,mean_fault_score,fars_total,fars_at_fault,fars_pct_at_fault,crss_total,crss_at_fault,...,accidents_per_million_miles,fars_rate_per_100_trucks,fars_rate_per_100_drivers,fars_rate_per_1m_miles,crss_rate_per_100_trucks,crss_rate_per_100_drivers,crss_rate_per_1m_miles,exposure_score,accident_exposure_score,safety_index
0,1,1,0,0.0,0.0,0,0,NaN,1,0,...,0.0,0.0,0.0,0.0,1.098901,25.0,142.857143,95.007000,0.0,100.0
1,10000,0,0,NaN,NaN,0,0,NaN,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2.009949,0.0,100.0
2,1000000,0,0,NaN,NaN,0,0,NaN,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.119162,0.0,100.0
3,1000002,0,0,NaN,NaN,0,0,NaN,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2.015922,0.0,100.0
4,1000004,0,0,NaN,NaN,0,0,NaN,0,0,...,NaN,0.0,0.0,NaN,0.000000,0.0,NaN,3.000000,0.0,100.0
